This notebook processes the `genosnp` file with data from the "Ice Age" paper into a sensible table, which can be used by other things than just ADMIXTOOLS etc.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

plt.style.use('ggplot')

### Read the table of individuals

In [4]:
!head ../raw_data/ice_age/individuals

In [5]:
individuals = pd.read_table('../raw_data/ice_age/individuals',
                            names=['sample_id', 'name', 'date',
                                     'in_table_s1', 'in_regression'],
                            skiprows=1, index_col=0)
individuals.head()

,sample_id,name,date,in_table_s1,in_regression
0,AG2,AfontovaGora2,16710,1,1
1,AfontovaGora3,AfontovaGora3,17000,1,1
2,Altai,Altai,70000,0,0
3,B_Australian-4,Australian,0,1,0
4,Chimp,Chimp,0,0,0


Replace unallowed characters with underscores:

In [6]:
individuals.sample_id = individuals.sample_id.apply(lambda s: s.replace('-', '_'))

### Read the positions of sites from the archaic admixture array

In [7]:
array_sites = pd.read_table('../raw_data/ice_age/filter.gz', names=['chrom', 'end', 'set_1', 'set_2'])

In [8]:
array_sites.head()

,chrom,end,set_1,set_2
0,1,847983,1,1
1,1,853089,1,1
2,1,853596,1,1
3,1,854793,1,1
4,1,858862,0,0


### Process the genosnp file

In [9]:
genosnp = pd.concat([pd.read_table('../raw_data/ice_age/genosnp_first_500000.gz', sep=' ', header=None, na_values='-'),
                     pd.read_table('../raw_data/ice_age/genosnp_last_454456.gz', sep=' ', header=None, na_values='-')])
genosnp.columns = ['chrom', 'pos', 'id', 'genpos', 'allele0', 'allele1', 'bin', 'gt_string', 'bvalue']

In [10]:
genosnp.head()

,chrom,pos,id,genpos,allele0,allele1,bin,gt_string,bvalue
0,1,847983,1_847983,0.008480,T,C,8.0,901009000990900900000900000009099999909000010009,958
1,1,853089,1_853089,0.008531,C,G,8.0,991009000999990000900900000009099999909000990009,958
2,1,853596,1_853596,0.008536,G,A,8.0,901019000999990900990900009009999999909000990009,958
3,1,854793,1_854793,0.008548,C,A,8.0,991009000999990900900900009001099999909000990009,958
4,1,858862,1_858862,0.008589,C,G,7.0,990009000999990900990900900009099999909000000009,952


Convert the concatenated string of randomly called alleles to a DataFrame of integer values:

In [11]:
random_calls = pd.DataFrame([list(s) for s in list(genosnp.gt_string)], 
                            columns=individuals.sample_id.values, dtype=int,
                            index=genosnp.index)

In [12]:
random_calls.head()

,AG2,AfontovaGora3,Altai,B_Australian_4,Chimp,Continenza,B_Dai_4,Denisova,B_Dinka_3,Dolni13,...,Pavlov1,B_Sardinian_3,Stuttgart,Ust_Ishim,Villabruna,Vi_merge,B_Yoruba_3,Bichon,KK1,SATP
0,9,0,1,0,0,9,0,0,0,9,...,9,0,0,0,0,1,0,0,0,9
1,9,9,1,0,0,9,0,0,0,9,...,9,0,0,0,9,9,0,0,0,9
2,9,0,1,0,1,9,0,0,0,9,...,9,0,0,0,9,9,0,0,0,9
3,9,9,1,0,0,9,0,0,0,9,...,9,0,0,0,9,9,0,0,0,9
4,9,9,0,0,0,9,0,0,0,9,...,9,0,0,0,0,0,0,0,0,9


Combine both into one table of randomly called alleles and their positions, B-values, etc...:

In [13]:
snp_data = pd.concat([genosnp[['chrom', 'pos', 'bvalue']],
                      random_calls], axis=1, ignore_index=True)
snp_data.columns = ['chrom', 'pos', 'bvalue'] + list(individuals.sample_id)

Subset to the `783,974` sites used in the Ice Age paper:

In [14]:
snp_data = snp_data[(array_sites.set_1 == 1).values]

Subset to sites which have a B-value assigned:

In [15]:
snp_data = snp_data.dropna()

Check the result:

In [16]:
snp_data.head()

,chrom,pos,bvalue,AG2,AfontovaGora3,Altai,B_Australian_4,Chimp,Continenza,B_Dai_4,...,Pavlov1,B_Sardinian_3,Stuttgart,Ust_Ishim,Villabruna,Vi_merge,B_Yoruba_3,Bichon,KK1,SATP
0,1,847983,958,9,0,1,0,0,9,0,...,9,0,0,0,0,1,0,0,0,9
1,1,853089,958,9,9,1,0,0,9,0,...,9,0,0,0,9,9,0,0,0,9
2,1,853596,958,9,0,1,0,1,9,0,...,9,0,0,0,9,9,0,0,0,9
3,1,854793,958,9,9,1,0,0,9,0,...,9,0,0,0,9,9,0,0,0,9
5,1,867151,946,9,9,1,0,9,9,0,...,9,0,0,0,0,9,0,0,0,0


### Add distance to the nearest exon

In [17]:
from pybedtools import BedTool

Download the genome annotations and subset to exons only:

In [18]:
gtf = pd.read_table('ftp://ftp.ensembl.org/pub/release-75/gtf/homo_sapiens/Homo_sapiens.GRCh37.75.gtf.gz',
                    header=None, sep='\t', skipinitialspace=True, skiprows=5, compression='gzip',
                    names=['chrom', 'source', 'feature', 'start', 'end',
                           'score', 'strand', 'frame', 'attribute'], low_memory=False)

In [19]:
gtf.head()

,chrom,source,feature,start,end,score,strand,frame,attribute
0,1,pseudogene,gene,11869,14412,.,+,.,"gene_id ""ENSG00000223972""; gene_name ""DDX11L1""..."
1,1,processed_transcript,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972""; transcript_id ""ENST..."
2,1,processed_transcript,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972""; transcript_id ""ENST..."
3,1,processed_transcript,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972""; transcript_id ""ENST..."
4,1,processed_transcript,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972""; transcript_id ""ENST..."


In [20]:
gtf.feature.value_counts()

exon              1306656
CDS                791856
UTR                304070
transcript         215170
stop_codon          73411
start_codon         73358
gene                63677
Selenocysteine        114
Name: feature, dtype: int64

In [21]:
exons = BedTool.from_dataframe(
    gtf[(gtf.source == "protein_coding") &
        (gtf.feature == "exon")]
).sort().merge()

Find the distance of each site from the admixture array to the nearest exon:

In [22]:
snp_data['start'] = snp_data.pos - 1
snp_data['end'] = snp_data.pos

In [23]:
distances = BedTool.from_dataframe(snp_data[['chrom', 'start', 'end']]).sort() \
                .closest(exons, t='first', d=True).to_dataframe()[[-1]].values
snp_data['exon_distance'] = distances

### Output the processed genosnp table in a tab-separated file

In [24]:
!mkdir -p ../clean_data

In [25]:
snp_data[['chrom', 'pos', 'bvalue', 'exon_distance'] +
         list(individuals.sample_id)].to_csv('../clean_data/ice_age.tsv', sep='\t', index=False)